In [1]:
#imports
import requests
import pandas as pd
import os
from datetime import datetime

## config


In [2]:
# set your data folder
desti_folder = './data'

# give your socrata token
token = 'YOURTOKENHERE'


In [7]:
crime_start_date = '2019-01-01'
crime_end_date = '2023-12-31'

In [8]:
envcomplaint_start_date = '2019-01-01'
envcomplaint_end_date = '2023-12-31'

In [3]:
os.makedirs(desti_folder, exist_ok=True )

In [4]:
timestamp = datetime.now()
timestamp = timestamp.strftime("%Y-%m-%d-%H-%M-%S")

In [5]:
def get_data(name, db, maxrows, parameters):
    # some key info
    db_id = db
    limit = maxrows
    i = 0
    offset_counter = 0
    pickle = True
    data_name = name
    # dict for logging
    log_dict = {}

    # base url for the request
    baseurl = ("https://data.cityofchicago.org/resource/"
            f"{db_id}.json?")

    while pickle:
        #set our params
        # note that you have to URL econde 
        params = parameters + (
            f"&$limit={limit}"
            f"&$offset={offset_counter}"
            #f"&$$app_token={token}"
        )
        
        # make the request 
        data_req = requests.get(baseurl+params)

        # convert to json
        data_json = data_req.json()

        # make it a df
        df = pd.json_normalize(data_json)

        # write df
        
        
        # log
        minilog = {}
        minilog['records'] = df.shape[0]
        minilog['chunk'] = i
        minilog['status'] = data_req.status_code
        minilog['offset'] = offset_counter
        log_dict[f'run_{i}'] = minilog
        log_df = pd.DataFrame.from_dict(log_dict,orient='index')
        
        if df.shape[0] == 0:
            break
        else:
            df.to_json(f'{desti_folder}/{data_name}_chunk_{i}.json',orient='records', lines=True)
            log_df.to_csv(f'{desti_folder}/{data_name}_{timestamp}_log.csv')

        # increment the chunk count
        print(i)
        i += 1

        # increment the offset
        offset_counter += limit

        # check if we need to end the loop
        if df.shape[0] == 0:
            pickle = False
        elif data_req.status_code == 200:
            pickle = True
        else: 
            pickle = False
            print(data_req.status_code)



In [9]:
crime_params = (
            f"$where=DATE%20BETWEEN%20%27{crime_start_date}T00%3A00%3A00.000%27%20AND%20%27{crime_end_date}T23%3A59%3A59.000%27"
            f"&$order=DATE"
            #f"&$$app_token={token}"
            )

get_data('crime5', 'ijzp-q8t2', 500000, crime_params)


env_params = (
            f"$where=complaint_date%20BETWEEN%20%27{envcomplaint_start_date}T00%3A00%3A00.000%27%20AND%20%27{envcomplaint_end_date}T23%3A59%3A59.000%27"
            f"&$order=complaint_date"
            #f"&$$app_token={token}"
            )

get_data('envcomplaint5', 'fypr-ksnz', 500000, env_params)

offenders_params = (
    f"$where=block%20not%20like%20%27%25deport%25%27"
            #f"&$$app_token={token}"
            )

get_data('offenders', 'vc9r-bqvy', 500000, offenders_params)


0
1
2
0
0
